# Structured Agents with Reliable Tool Usage

This notebook demonstrates the improved agent system (v2) that uses structured responses to ensure reliable tool usage across different LLM providers.

In [1]:
# Enable async support in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Import the structured agent system
from agent_system_v2 import StructuredAgent, Tool
from llm_providers import create_llm, create_llm_for_use_case, RECOMMENDED_MODELS
import asyncio
import json
import os

## Key Differences in V2

The structured agent system (v2) improves on v1 by:

1. **Structured JSON Responses** - Agents respond with clear JSON containing thought/action/input
2. **Reliable Tool Execution** - Tools are used when requested, not just mentioned
3. **Iterative Processing** - Can perform multiple actions to complete complex tasks
4. **Better Error Handling** - Gracefully handles JSON parsing failures

In [2]:
# Check available providers
available_providers = []

if os.getenv('OPENAI_API_KEY'):
    available_providers.append('openai')
    print("✓ OpenAI API key found")

if os.getenv('ANTHROPIC_API_KEY'):
    available_providers.append('anthropic')
    print("✓ Anthropic API key found")

if os.getenv('GOOGLE_API_KEY'):
    available_providers.append('gemini')
    print("✓ Google API key found")

✓ OpenAI API key found
✓ Anthropic API key found
✓ Google API key found


## 1. Basic Tool Usage - Calculator Example

In [3]:
# Create a calculator tool
def calculator(expression: str) -> str:
    """Evaluates mathematical expressions"""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

calc_tool = Tool(
    name="calculator",
    func=calculator,
    description="Evaluates mathematical expressions accurately"
)

# Test with different providers
test_task = "Please use the calculator to compute 15 * 23 + 42"

for provider in available_providers:
    print(f"\n{'='*50}")
    print(f"Testing {provider} with structured agent")
    print(f"{'='*50}")
    
    agent = StructuredAgent(
        name=f"{provider}_calculator",
        context={"role": "math assistant"},
        tools=[calc_tool],
        llm_config={"provider": provider, "temperature": 0.7}
    )
    
    result = await agent.run(test_task)
    
    print(f"Tools used: {result['tools_used']}")
    print(f"Response: {result['response'][:150]}...")


Testing openai with structured agent
Tools used: ['calculator']
Response: The result of computing 15 * 23 + 42 is 387....

Testing anthropic with structured agent
Tools used: ['calculator']
Response: Here is the response to the original task:

Using the calculator, 15 * 23 + 42 equals 387.

To break it down:
15 * 23 = 345
345 + 42 = 387

Therefore,...

Testing gemini with structured agent
Tools used: ['calculator']
Response: 15 * 23 + 42 = 387...


## 2. Multiple Tools - Complex Tasks

In [ ]:
# Create multiple tools
def string_manipulator(action_and_text: str) -> str:
    """Manipulates strings. Format: 'action:text' where action is reverse/uppercase/lowercase"""
    try:
        action, text = action_and_text.split(':', 1)
        if action == "reverse":
            return text[::-1]
        elif action == "uppercase":
            return text.upper()
        elif action == "lowercase":
            return text.lower()
        else:
            return f"Unknown action: {action}"
    except:
        return "Error: Use format 'action:text'"

def word_counter(text: str) -> str:
    """Counts words in text"""
    words = text.split()
    return f"Word count: {len(words)}"

string_tool = Tool(
    name="string_manipulator",
    func=string_manipulator,
    description="Manipulates strings (reverse/uppercase/lowercase)"
)

count_tool = Tool(
    name="word_counter",
    func=word_counter,
    description="Counts words in text"
)

# Create agent with multiple tools
multi_tool_agent = StructuredAgent(
    name="TextProcessor",
    context={"role": "text processing assistant"},
    tools=[string_tool, count_tool, calc_tool],
    llm_config={"provider": available_providers[0]}
)

# Test multiple tool usage
tasks = [
    "Use the string_manipulator to reverse the text 'Hello World'",
    "Count the words in 'The quick brown fox jumps over the lazy dog' using word_counter",
    "First reverse 'OpenAI' using string_manipulator, then count its letters"
]

for task in tasks:
    print(f"\nTask: {task}")
    result = await multi_tool_agent.run(task)
    print(f"Tools used: {result['tools_used']}")
    print(f"Response: {result['response'][:200]}...")

## 3. Agent as a Tool - Hierarchical Systems

In [ ]:
# Create specialized agents
math_agent = StructuredAgent(
    name="MathExpert",
    context={"specialty": "complex mathematical computations"},
    tools=[calc_tool],
    llm_config={"provider": available_providers[0]}
)

text_agent = StructuredAgent(
    name="TextExpert",
    context={"specialty": "text analysis and manipulation"},
    tools=[string_tool, count_tool],
    llm_config={"provider": available_providers[0]}
)

# Create tools from agents
async def math_agent_tool(task: str) -> str:
    result = await math_agent.run(task)
    return result['response']

async def text_agent_tool(task: str) -> str:
    result = await text_agent.run(task)
    return result['response']

math_tool = Tool(
    name="math_expert",
    func=math_agent_tool,
    description="Delegate mathematical tasks to the math expert"
)

text_tool = Tool(
    name="text_expert",
    func=text_agent_tool,
    description="Delegate text processing tasks to the text expert"
)

# Create a manager agent
manager = StructuredAgent(
    name="Manager",
    context={"role": "coordinate specialist agents"},
    tools=[math_tool, text_tool],
    llm_config={"provider": available_providers[0]}
)

# Test delegation
manager_task = "Use the math_expert to calculate 123 * 456, then use the text_expert to reverse the result"
result = await manager.run(manager_task)

print(f"Manager used tools: {result['tools_used']}")
print(f"Manager response: {result['response']}")

## 4. Comparing Structured vs Original Agents

In [ ]:
# Import both versions
from agent_system import Agent as OriginalAgent
from agent_system_v2 import StructuredAgent

# Same task and tool
comparison_task = "Use the calculator to find 99 * 99"

print("Testing Original Agent:")
original = OriginalAgent(
    name="Original",
    tools=[calc_tool],
    llm_config={"provider": available_providers[0]}
)
orig_result = await original.run(comparison_task)
print(f"  Tools used: {orig_result['tools_used']}")
print(f"  Response: {orig_result['response'][:100]}...")

print("\nTesting Structured Agent:")
structured = StructuredAgent(
    name="Structured",
    tools=[calc_tool],
    llm_config={"provider": available_providers[0]}
)
struct_result = await structured.run(comparison_task)
print(f"  Tools used: {struct_result['tools_used']}")
print(f"  Response: {struct_result['response'][:100]}...")

## 5. Custom Tool Creation

In [ ]:
# Create a custom tool that simulates API calls
def weather_api(location: str) -> str:
    """Simulates weather API calls"""
    # Mock weather data
    weather_data = {
        "new york": "Sunny, 72°F",
        "london": "Cloudy, 59°F",
        "tokyo": "Rainy, 65°F",
        "paris": "Partly cloudy, 68°F"
    }
    
    location_lower = location.lower().strip()
    if location_lower in weather_data:
        return f"Weather in {location}: {weather_data[location_lower]}"
    else:
        return f"Weather data not available for {location}"

weather_tool = Tool(
    name="weather_api",
    func=weather_api,
    description="Gets current weather for a location"
)

# Create a tool that needs specific formatting
def database_query(query: str) -> str:
    """Simulates database queries. Format: 'table:condition'"""
    try:
        table, condition = query.split(':', 1)
        # Mock database results
        mock_data = {
            "users": "Found 42 users matching condition",
            "orders": "Found 156 orders matching condition",
            "products": "Found 89 products matching condition"
        }
        
        if table in mock_data:
            return f"{mock_data[table]}: {condition}"
        else:
            return f"Table '{table}' not found"
    except:
        return "Error: Use format 'table:condition'"

db_tool = Tool(
    name="database",
    func=database_query,
    description="Queries database. Format: 'table:condition'"
)

# Create agent with custom tools
custom_agent = StructuredAgent(
    name="DataAssistant",
    context={"role": "data and information retrieval"},
    tools=[weather_tool, db_tool],
    llm_config={"provider": available_providers[0]}
)

# Test custom tools
custom_tasks = [
    "Use weather_api to get the weather in Tokyo",
    "Query the database for users with age > 25 using the database tool",
    "First get weather for London, then query orders from the last week"
]

for task in custom_tasks:
    print(f"\nTask: {task}")
    result = await custom_agent.run(task)
    print(f"Tools used: {result['tools_used']}")
    print(f"Response: {result['response']}")

## 6. Error Handling and Edge Cases

In [ ]:
# Test error handling
error_tasks = [
    "Use calculator to compute 'invalid expression'",
    "Use a non-existent tool called magic_wand",
    "Use string_manipulator with wrong format"
]

error_agent = StructuredAgent(
    name="ErrorTest",
    tools=[calc_tool, string_tool],
    llm_config={"provider": available_providers[0]}
)

for task in error_tasks:
    print(f"\nTask: {task}")
    result = await error_agent.run(task)
    print(f"Tools used: {result['tools_used']}")
    print(f"Response: {result['response'][:200]}...")

## 7. Performance Comparison Across Providers

In [ ]:
# Create same agent with different providers
import time

performance_task = "Use the calculator to compute 1234 * 5678"
performance_results = {}

for provider in available_providers:
    print(f"\nTesting {provider}...")
    
    agent = StructuredAgent(
        name=f"{provider}_perf_test",
        tools=[calc_tool],
        llm_config={
            "provider": provider,
            "model": RECOMMENDED_MODELS['fast'][provider]  # Use fast models
        }
    )
    
    start_time = time.time()
    result = await agent.run(performance_task)
    end_time = time.time()
    
    performance_results[provider] = {
        "time": end_time - start_time,
        "tools_used": result['tools_used'],
        "success": len(result['tools_used']) > 0
    }

# Show results
print("\nPerformance Summary:")
print(f"{'Provider':<15} {'Time (s)':<10} {'Tool Used':<10}")
print("-" * 35)
for provider, stats in performance_results.items():
    print(f"{provider:<15} {stats['time']:<10.2f} {'✓' if stats['success'] else '✗':<10}")

## Key Improvements in Structured Agents

The structured agent system provides:

1. **Reliable Tool Usage** - Tools are actually executed when mentioned
2. **Structured Responses** - Clear JSON format for thoughts and actions
3. **Better Error Handling** - Graceful fallbacks for parsing failures
4. **Provider Compatibility** - Works consistently across OpenAI, Anthropic, and Gemini
5. **Iterative Processing** - Can perform multiple actions if needed

### When to Use Structured Agents

Use structured agents when:
- You need reliable tool execution
- You want consistent behavior across providers
- You're building complex multi-agent systems
- You need clear action tracking and debugging

### Migration from V1

To migrate from the original agent system:
1. Replace `Agent` with `StructuredAgent`
2. No other changes needed - same API
3. Tools and context work the same way